In [16]:
import os
from tqdm import tqdm
import yt_dlp
import cv2
import pandas as pd
import time
import random

In [ ]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/AndresR2909/youtube_transcriptions_train_dataset_2025_gpt4.1/data/train-00000-of-00001.parquet")

In [8]:
# ----------- ELIMINAR CANAL USACRYPTONOTICIAS -----------
print("Cantidad de filas antes de eliminar USACRYPTONOTICIAS:", len(df))
print("\nCanales antes de filtrar:")
print(df['channel_name'].value_counts())

# Eliminar filas del canal USACRYPTONOTICIAS
df_filtered = df[df['channel_name'] != 'USACRYPTONOTICIAS'].copy()

print(f"\nCantidad de filas después de eliminar USACRYPTONOTICIAS: {len(df_filtered)}")
print("\nCanales después de filtrar:")
print(df_filtered['channel_name'].value_counts())


Cantidad de filas antes de eliminar USACRYPTONOTICIAS: 1304

Canales antes de filtrar:
channel_name
Esteban Perez                     715
Bolsas hoy | Invierte y Crece     237
USACRYPTONOTICIAS                 170
ARENA ALFA                        125
Bitcoin hoy                        45
Bolsas hoy | Esteban Pérez         12
Name: count, dtype: int64

Cantidad de filas después de eliminar USACRYPTONOTICIAS: 1134

Canales después de filtrar:
channel_name
Esteban Perez                     715
Bolsas hoy | Invierte y Crece     237
ARENA ALFA                        125
Bitcoin hoy                        45
Bolsas hoy | Esteban Pérez         12
Name: count, dtype: int64


In [9]:
df_filtered.head()

,channel_name,video_id,publish_date,duration,text,summary,train_text,num_tokens
0,Bolsas hoy | Invierte y Crece,MEFcIWdwoEI,None,822.0,sp500 y oro alcanzan sus objetivos proyectados...,**Informe de Análisis de Mercados Financieros*...,"Below is an instruction that describes a task,...",4165
1,Bolsas hoy | Invierte y Crece,yQdTS2296V8,None,1251.0,la fed te hace perder dinero porque no puede p...,- **Introducción:** \nEl texto analiza la sit...,"Below is an instruction that describes a task,...",5320
2,Bolsas hoy | Invierte y Crece,5uDK19-gLkw,None,907.0,máximos históricos del sp 500 son una cuestión...,- **Introducción:** \nEl texto proporciona un...,"Below is an instruction that describes a task,...",4443
3,Bolsas hoy | Invierte y Crece,dm-GKckZQG4,None,1142.0,mejor aprender a saber qué hacer en vez de bus...,- **Introducción:** \nEl texto analizado corr...,"Below is an instruction that describes a task,...",5080
4,Bolsas hoy | Invierte y Crece,n4ZxNi2TjaY,None,1055.0,nuevas proyecciones ya que los planes se han e...,**Informe de Análisis de Mercados Financieros ...,"Below is an instruction that describes a task,...",4991


In [17]:
#df_filtered.to_csv("datasets/ds_links_videos_youtube.csv",index=False)
df_filtered = pd.read_csv("datasets/ds_links_videos_youtube.csv")

In [18]:
# ----------- CREAR LISTA DE LINKS CON VIDEO_ID -----------
# Obtener los video_id únicos del DataFrame filtrado
video_ids = df_filtered['video_id'].unique()

# Crear la lista de links de YouTube
youtube_links = [f"https://www.youtube.com/watch?v={video_id}" for video_id in video_ids]

In [29]:
# ----------- CONFIGURACIÓN ----------
VIDEO_LINKS = youtube_links
DOWNLOADS_DIR = 'videos_youtube'
FRAMES_DIR = 'frames'   # carpeta donde se guardan frames
FRAME_INTERVAL_SEC = 2   # saca un frame cada 2 segundos

os.makedirs(DOWNLOADS_DIR, exist_ok=True)
os.makedirs(FRAMES_DIR, exist_ok=True)

In [20]:
# ----------- SOLUCIÓN PARA ERRORES DE DESCARGA -----------

def download_videos_robust(video_urls, output_dir=DOWNLOADS_DIR, max_retries=3):
    """
    Función mejorada para descargar videos con manejo de errores y reintentos
    """
    successful_downloads = []
    failed_downloads = []
    
    for i, url in enumerate(video_urls):
        print(f"\n--- Descargando video {i+1}/{len(video_urls)} ---")
        print(f"URL: {url}")
        
        for attempt in range(max_retries):
            try:
                ydl_opts = {
                    'outtmpl': os.path.join(output_dir+'/'+'%(uploader)s', '%(id)s_%(upload_date)s.%(ext)s'),
                    'format': '135/136/137',#best[height<=720]',#'18/22/137/136/135/134/133/160/bestvideo+bestaudio/best',
                    'noplaylist': True,
                    'quiet': False,
                    'retries': 3,
                    'fragment_retries': 3,
                    'socket_timeout': 30,
                    'http_chunk_size': 10485760,  # 10MB chunks
                }
                
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([url])
                
                print(f"✅ Descarga exitosa: {url}")
                successful_downloads.append(url)
                break
                
            except Exception as e:
                print(f"❌ Intento {attempt + 1} falló: {str(e)}")
                if attempt < max_retries - 1:
                    wait_time = random.uniform(5, 15)  # Espera aleatoria entre 5-15 segundos
                    print(f"⏳ Esperando {wait_time:.1f} segundos antes del siguiente intento...")
                    time.sleep(wait_time)
                else:
                    print(f"💥 Falló definitivamente: {url}")
                    failed_downloads.append(url)
    
    print(f"\n📊 RESUMEN DE DESCARGAS:")
    print(f"✅ Exitosas: {len(successful_downloads)}")
    print(f"❌ Fallidas: {len(failed_downloads)}")
    
    if failed_downloads:
        print(f"\n🔴 Videos que fallaron:")
        for url in failed_downloads:
            print(f"  - {url}")
    
    return successful_downloads, failed_downloads


In [ ]:
successful, failed = download_videos_robust(video_urls=VIDEO_LINKS, output_dir=DOWNLOADS_DIR)

In [ ]:
links_successful = pd.DataFrame(successful, columns=['url_successful'])
links_successful.head()

links_failed = pd.DataFrame((failed), columns=['url_failed'])
links_successful.to_csv('links_successful.csv', index=False)
links_failed.to_csv('links_failed.csv', index=False)
links_failed.head()

In [27]:
links_failed = pd.read_csv('links_failed.csv')
failed=links_failed["url_failed"].to_list()

In [33]:
# ----------- 2. EXTRAER FRAMES DE LOS VIDEOS -----------
def extract_frames_from_video(video_path, frames_dir, step_sec=2):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    
    frame_paths = []
    success, image = cap.read()
    count = 0
    sec = 0
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    while success:
        if int(cap.get(cv2.CAP_PROP_POS_MSEC)) >= sec*1000:
            frame_path = os.path.join(frames_dir, f"{base_name}_frame{count}.jpg")
            cv2.imwrite(frame_path, image)
            frame_paths.append(frame_path)
            sec += step_sec
        success, image = cap.read()
        count += 1
    cap.release()
    return duration, frame_paths

In [34]:
archivos_filtrados = list(filter(lambda x: x != '.DS_Store', os.listdir(DOWNLOADS_DIR)))
print(f"\nTotal de archivos: {len(archivos_filtrados)}")



Total de archivos: 3


In [35]:
import re
patron = r'^(.+)_(\d{8})\.mp4$'
    
# EXTRAER TODOS LOS FRAMES (esto puede tomar tiempo la primera vez)
eda_data = []
for channel in tqdm(archivos_filtrados):
    path_channel = DOWNLOADS_DIR+"/"+channel
    path_frames = FRAMES_DIR+"/"+channel
    os.makedirs(path_frames, exist_ok=True)
    for video in tqdm(os.listdir(path_channel)):
        if video.endswith('.mp4'):
            video_path = os.path.join(path_channel, video)
            duration, frame_paths = extract_frames_from_video(video_path, path_frames, FRAME_INTERVAL_SEC)
            match = re.match(patron, video)
            video_id = match.group(1)
            fecha = match.group(2)
            eda_data.append({
                'video_id': video_id,
                'channel': channel,
                'video_date': fecha,
                'video_filename': video,
                'duration': duration,
                'n_frames': len(frame_paths),
                'frame_paths': frame_paths
            })

df_videos = pd.DataFrame(eda_data)
df_videos

100%|██████████| 3/3 [7:10:05<00:00, 8601.79s/it]  


,video_id,channel,video_date,video_filename,duration,n_frames,frame_paths
0,tKBpmMwwJaE,Bitcoin hoy ｜ Esteban Perez Trader,20220130,tKBpmMwwJaE_20220130.mp4,1019.333333,510,[frames/Bitcoin hoy ｜ Esteban Perez Trader/tKB...
1,OHqn_ak8cX0,Bitcoin hoy ｜ Esteban Perez Trader,20220829,OHqn_ak8cX0_20220829.mp4,792.666667,397,[frames/Bitcoin hoy ｜ Esteban Perez Trader/OHq...
2,wVKhJhsQfT4,Bitcoin hoy ｜ Esteban Perez Trader,20231207,wVKhJhsQfT4_20231207.mp4,493.566667,247,[frames/Bitcoin hoy ｜ Esteban Perez Trader/wVK...
3,-7ju1t4k4oE,Bitcoin hoy ｜ Esteban Perez Trader,20230411,-7ju1t4k4oE_20230411.mp4,1279.833333,640,[frames/Bitcoin hoy ｜ Esteban Perez Trader/-7j...
4,IRAij7lx8xk,Bitcoin hoy ｜ Esteban Perez Trader,20221206,IRAij7lx8xk_20221206.mp4,961.500000,481,[frames/Bitcoin hoy ｜ Esteban Perez Trader/IRA...
...,...,...,...,...,...,...,...
1128,KfALD5wJIsM,Bolsas hoy ｜ Esteban Pérez Inversor,20240229,KfALD5wJIsM_20240229.mp4,1204.436567,603,[frames/Bolsas hoy ｜ Esteban Pérez Inversor/Kf...
1129,CDjcddrk-ms,Bolsas hoy ｜ Esteban Pérez Inversor,20240115,CDjcddrk-ms_20240115.mp4,484.266667,243,[frames/Bolsas hoy ｜ Esteban Pérez Inversor/CD...
1130,IY8dYm5DoG4,Bolsas hoy ｜ Esteban Pérez Inversor,20240516,IY8dYm5DoG4_20240516.mp4,1821.566667,911,[frames/Bolsas hoy ｜ Esteban Pérez Inversor/IY...
1131,VGdofDNzcss,Bolsas hoy ｜ Esteban Pérez Inversor,20240429,VGdofDNzcss_20240429.mp4,1916.133333,959,[frames/Bolsas hoy ｜ Esteban Pérez Inversor/VG...


In [36]:
df_videos.to_csv("datasets/df_videos_youtube_descargados.csv", index=False)